# **Supervised Learning code for Disaster Tweet Classification**

Installing the libraries


In [ ]:
!pip3 install ktrain==0.7.1

     |████████████████████████████████| 92kB 2.9MB/s 
     |████████████████████████████████| 6.7MB 10.3MB/s 
     |████████████████████████████████| 983kB 36.2MB/s 
     |████████████████████████████████| 245kB 38.8MB/s 
     |████████████████████████████████| 1.8MB 37.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.7.1-cp36-none-any.whl size=113531 sha256=8548fee9d7aee0e40e3d2b27a64568c0420311bc6fe101f5cbc27594bd0dcc46
  Stored in directory: /root/.cache/pip/wheels/b3/8b/ba/b274eb6f796b689e0eb2b2a05d88d1e1d66e5df88278aaf422
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-none-any.whl size=36139 sha256=70decc07cd26335be019d47e7c4b8f953a19e726a4a26ca86b3773c270f06f18
  Stored in directory: /root/.cache/pip/wheels/1f/59/04/12e95257aebfd27f7edaaf65ab7dd57b5f6cadfb183f1a4ccd
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=f8e395a9fb915da0e15b7907a5e7a5137a0d8ec55f3e09d333e8186fd12c8174
  Stored in directory: /root/.cache/

### Importing Libraries


In [ ]:
from sklearn.model_selection import train_test_split
import ktrain
from ktrain import text
import pandas as pd

### Reading training file

In [ ]:
train_df = pd.read_csv("train.csv")

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


### Splitting training data in to training and validation data

In [ ]:
random_seed = 12342
x_train, x_val, y_train, y_val = train_test_split(train_df['text'], train_df['target'], shuffle=True, test_size = 0.2, random_state=random_seed, stratify=train_df['target'])

### Convert data to features for BERT

In [ ]:
(x_train_bert,  y_train_bert), (x_val_bert, y_val_bert), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                                         x_test = x_val, y_test=y_val,
                                                                                          class_names= ["0", "1"],
                                                                                          preprocess_mode='bert',
                                                                                          lang = 'en',
                                                                                          maxlen=65, 
                                                                                          max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


### Create Model Instance and train it for 6 epochs




In [ ]:
model = text.text_classifier('bert', train_data=(x_train_bert, y_train_bert), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train_bert, y_train_bert), val_data=(x_val_bert, y_val_bert), batch_size=16)

Is Multi-Label? False
maxlen is 65
done.


In [ ]:
# train the model for 5 epochs
learner.fit_onecycle(1e-5, 5)



begin training using onecycle policy with max lr of 1e-05...
Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 193s 32ms/sample - loss: 0.5283 - acc: 0.7450 - val_loss: 0.4063 - val_acc: 0.8339
Epoch 2/5
6090/6090 [==============================] - 193s 32ms/sample - loss: 0.3862 - acc: 0.8333 - val_loss: 0.3836 - val_acc: 0.8385
Epoch 3/5
6090/6090 [==============================] - 193s 32ms/sample - loss: 0.3195 - acc: 0.8750 - val_loss: 0.4224 - val_acc: 0.8194
Epoch 4/5
6090/6090 [==============================] - 193s 32ms/sample - loss: 0.2245 - acc: 0.9181 - val_loss: 0.4566 - val_acc: 0.8345
Epoch 5/5
6090/6090 [==============================] - 193s 32ms/sample - loss: 0.1453 - acc: 0.9493 - val_loss: 0.5414 - val_acc: 0.8214


In [ ]:
learner.validate(val_data=(x_val_bert, y_val_bert), class_names=['No Disaster', 'Disaster'])

              precision    recall  f1-score   support

 No Disaster       0.84      0.86      0.85       869
    Disaster       0.80      0.78      0.79       654

    accuracy                           0.82      1523
   macro avg       0.82      0.82      0.82      1523
weighted avg       0.82      0.82      0.82      1523



array([[744, 125],
       [147, 507]])

### Getting predictor variable

In [ ]:
# getting predictor variable
predictor = ktrain.get_predictor(learner.model, preproc)

### Getting predictions on test data

In [ ]:
test_df = pd.read_csv("test.csv")
test_df["target"] = predictor.predict(test_df["text"].tolist())

test_df = test_df[["id", "target"]]


In [ ]:
test_df.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
test_df.to_csv("submisssions.csv", index=False)